In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import pandas as pd
import cv2

Création d'un gif (ou mp4) de l'ensemble des cellules d'une manip. 

In [ ]:

def create_trajectory_gif(DATA, pathway_experiment, experiment_name, pathway_saving,
                          first_frame=0, last_frame=240, dot_size=20, crop_size=None, target_particle=None):
    """
    Create an animated GIF of particle trajectories with history trails and optional cropping.

    Parameters:
    ----------
    DATA : pd.DataFrame
        DataFrame containing trajectory data, must include columns:
        'frame', 'particle', 'x', 'y'.
    pathway_experiment : str
        Path to the main folder containing experiment folders.
    experiment_name : str
        Name (or part of the name) of the experiment folder to search for.
    pathway_saving : str
        Path to save the resulting GIF and intermediary images.
    first_frame : int, optional
        First frame to start plotting.
    last_frame : int, optional
        Last frame to end plotting.
    dot_size : int, optional
        Size of the dots representing the particles.
    crop_size : int, optional
        Size of the crop around the target particle (in pixels). If None, no cropping is applied.
    target_particle : int, optional
        ID of the particle to center the crop on. Required if `crop_size` is specified.

    Returns:
    -------
    None
    """
    # Rechercher le dossier de l'expérience
    experiment_folder = None
    for folder in os.listdir(pathway_experiment):
        if experiment_name in folder:
            experiment_folder = os.path.join(pathway_experiment, folder, "mosaic")
            break

    if experiment_folder is None or not os.path.exists(experiment_folder):
        raise FileNotFoundError(f"Experiment folder containing '{experiment_name}' not found in {pathway_experiment}")

    # Importer les images du dossier 'mosaic'
    image_files = [os.path.join(experiment_folder, f) for f in sorted(os.listdir(experiment_folder)) if f.endswith((".png", ".jpg", ".tif"))]
    frames = image_files[first_frame:last_frame + 1]

    if len(frames) == 0:
        raise FileNotFoundError(f"No images found in {experiment_folder} for the specified frames.")

    # Créer le dossier de sauvegarde s'il n'existe pas
    pathway_saving_exp = os.path.join(pathway_saving, experiment_name)
    if not os.path.exists(pathway_saving_exp):
        os.makedirs(pathway_saving_exp)

    # Créer un dictionnaire de couleurs pour chaque particule
    particles = DATA['particle'].unique()
    colors = {particle: np.random.rand(3) for particle in particles}

    # Créer une liste pour stocker les chemins des images intermédiaires
    image_paths = []

    # Traçage des trajectoires pour chaque frame
    for frame_idx, frame_path in enumerate(frames):
        current_frame = first_frame + frame_idx
        frame_data = DATA[DATA['frame'] <= current_frame]

        # Charger l'image de fond
        background = plt.imread(frame_path)

        # Préparer la figure
        plt.figure(figsize=(20, 20))
        plt.imshow(background, cmap='gray')
        plt.axis('off')

        # Tracer les trajectoires complètes pour chaque particule
        for particle, color in colors.items():
            particle_data = frame_data[frame_data['particle'] == particle]
            if not particle_data.empty:
                x_data, y_data = particle_data['x'].values, particle_data['y'].values
                plt.plot(
                    x_data, y_data,
                    color=color, alpha=0.6, linewidth=1, marker='.', markersize=dot_size
                )

        # Sauvegarder l'image intermédiaire
        image_path = os.path.join(pathway_saving_exp, f"frame_{current_frame:04d}.png")
        plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
        plt.close()
        image_paths.append(image_path)

    # Créer le GIF
    gif_path = os.path.join(pathway_saving_exp, f"trajectories_particle_{target_particle or 'all'}.gif")
    with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:
        for image_path in image_paths:
            writer.append_data(imageio.imread(image_path))

    print(f"GIF saved to {gif_path}")

In [ ]:
create_trajectory_gif(
    DATA=DATA,
    pathway_experiment="/Users/souchaud/Desktop/A_analyser/CytoOne_SorC_10x_faits/",
    experiment_name="ASMOT211",
    pathway_saving="/Users/souchaud/Desktop/Analyses/gif",
    first_frame=0,
    last_frame=240,
    dot_size=1
)

Création d'un gif ou mp4 d'une cellule bien ciblée. Seul le point du tracking à chaque image est tracé, toute la trajectoires n'est pas tracée au fur et à mesure

In [ ]:

def create_trajectory_gif_particle_fixed_frame(data, target_particle, pathway_experiment, pathway_saving,
                                               first_frame=0, last_frame=240, dot_size=2, crop_size=None):
    """
    Create an animated GIF of particle trajectories with a fixed cropping frame.

    Parameters:
    ----------
    data : pd.DataFrame
        DataFrame containing trajectory data, must include columns:
        'frame', 'particle', 'x', 'y'.
    pathway_experiment : str
        Path to the main folder containing experiment folders.
    pathway_saving : str
        Path to save the resulting GIF and intermediary images.
    first_frame : int, optional
        First frame to start plotting.
    last_frame : int, optional
        Last frame to end plotting.
    dot_size : int, optional
        Size of the dots representing the particles.
    crop_size : int, optional
        Size of the crop around the target particle (in pixels). Required.
    target_particle : int, optional
        ID of the particle to center the crop on.

    Returns:
    -------
    None
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Les données doivent être un DataFrame Pandas.")
    if target_particle is None:
        raise ValueError("Vous devez spécifier une particule cible ('target_particle').")
    if crop_size is None:
        raise ValueError("Vous devez spécifier une taille de recadrage ('crop_size').")
    if 'experiment' not in data.columns or 'particle' not in data.columns:
        raise ValueError("Les colonnes 'experiment' et 'particle' doivent exister dans les données.")

    # Isolation des données pour la particule cible
    particle_data = data[data['particle'] == target_particle]
    if particle_data.empty:
        raise ValueError(f"La particule cible {target_particle} n'existe pas dans les données.")

    # Extraction du nom de l'expérience
    experiment_name = particle_data['experiment'].iloc[0]
    print(f"Nom de l'expérience détecté : {experiment_name}")

    # Rechercher le dossier de l'expérience
    experiment_folder = None
    for folder in os.listdir(pathway_experiment):
        if experiment_name in folder:
            experiment_folder = os.path.join(pathway_experiment, folder, "mosaic")
            break

    if experiment_folder is None or not os.path.exists(experiment_folder):
        raise FileNotFoundError(f"Experiment folder containing '{experiment_name}' not found in {pathway_experiment}")

    # Importer les images du dossier 'mosaic'
    image_files = [os.path.join(experiment_folder, f) for f in sorted(os.listdir(experiment_folder)) if f.endswith((".png", ".jpg", ".tif"))]
    frames = image_files[first_frame:last_frame + 1]

    if len(frames) == 0:
        raise FileNotFoundError(f"No images found in {experiment_folder} for the specified frames.")

    # Créer le dossier de sauvegarde s'il n'existe pas
    pathway_saving_exp = os.path.join(pathway_saving, experiment_name) + str(target_particle)
    if not os.path.exists(pathway_saving_exp):
        os.makedirs(pathway_saving_exp)

    # Définir les coordonnées de recadrage fixes à partir de la position initiale de la particule
    frame_data_init = particle_data.iloc[0]
    x_0, y_0 = int(frame_data_init['x']), int(frame_data_init['y'])

    left = max(0, x_0 - crop_size // 2)
    upper = max(0, y_0 - crop_size // 2)
    right = left + crop_size
    lower = upper + crop_size

    # Assurez-vous que les coordonnées restent dans les limites des images
    for frame_number, name_frame in enumerate(frames):
        with Image.open(name_frame) as img:
            img_width, img_height = img.size
            right = min(right, img_width)
            lower = min(lower, img_height)

            cropped_img = img.crop((left, upper, right, lower))

            # Données pour le tracking
            frame_data = particle_data[particle_data['frame'] == (first_frame + frame_number)]
            if frame_data.empty:
                continue
            x, y = int(frame_data['x'].iloc[0]), int(frame_data['y'].iloc[0])
            rel_x, rel_y = x - left, y - upper

            # Création de l'image avec le tracking
            fig, ax = plt.subplots(figsize=(20, 20))
            ax.imshow(cropped_img, cmap='gray')
            ax.plot(rel_x, rel_y, 'ro', markersize=dot_size)
            ax.axis('off')

            # Sauvegarde de l'image
            cropped_img_path = os.path.join(pathway_saving_exp, f"frame_{frame_number:04d}.png")
            plt.savefig(cropped_img_path, bbox_inches='tight')
            plt.close(fig)

    # Création du GIF
    # gif_path = os.path.join(pathway_saving_exp, f"trajectory_particle_{target_particle}.gif")
    # with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:
    #     for frame_number in range(len(frames)):
    #         frame_path = os.path.join(pathway_saving_exp, f"frame_{frame_number:04d}.png")
    #         writer.append_data(imageio.imread(frame_path))
    # print(f"GIF sauvegardé à : {gif_path}")

    # Création video MP4
        # Création de la vidéo MP4
    video_path = os.path.join(pathway_saving_exp, f"trajectory_particle_{target_particle}.mp4")
    
    # Charger la première image pour déterminer les dimensions
    first_frame_path = os.path.join(pathway_saving_exp, "frame_0000.png")
    if not os.path.exists(first_frame_path):
        raise FileNotFoundError("Le premier cadre n'a pas été généré.")
    
    first_frame = imageio.imread(first_frame_path)
    height, width, layers = first_frame.shape
    
    # Initialisation du writer vidéo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour MP4
    video_writer = cv2.VideoWriter(video_path, fourcc, 10, (width, height))  # 10 fps
    
    # Ajouter chaque image à la vidéo
    for frame_number in range(len(frames)):
        frame_path = os.path.join(pathway_saving_exp, f"frame_{frame_number:04d}.png")
        if os.path.exists(frame_path):
            frame = cv2.imread(frame_path)
            video_writer.write(frame)
    
    # Libération du writer
    video_writer.release()
    print(f"Vidéo MP4 sauvegardée à : {video_path}")

In [ ]:
create_trajectory_gif_particle_fixed_frame(
    data=DATA,
    target_particle=2,
    pathway_experiment="/Users/souchaud/Desktop/A_analyser/CytoOne_SorC_10x_faits/",
    pathway_saving="/Users/souchaud/Desktop/Analyses/gif",
    first_frame=0,
    last_frame=185,
    dot_size=25,
    crop_size=200  # Taille du cadre fixe
)